In [ ]:
import sys # required for relative imports in jupyter lab
sys.path.insert(0, '../') 

from cosmosis.model import tv_model
from cosmosis.learning import Learn, Selector
from cosmosis.dataset import LoadImage, AsTensor

from dataset import Ranzcr

from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms

from IPython.display import display


In [ ]:
#TODO turn targets into onehots also with none category
#create models for each target_type
#accuracy metric AOC for each category averaged see kaggle
#dataset normalization

ds_params={'train_params': {'embed': [],
                            'image_dir': './data/ranzcr/train',
                            'targets_csv': './data/ranzcr/train.csv',
                            'target_type': 'ETT',
                            'transform': transforms.Compose([LoadImage(),
                                                             transforms.RandomRotation(10),
                                                             transforms.Resize(1500),
                                                             transforms.CenterCrop(1200),
                                                             transforms.Resize(300),
                                                             transforms.RandomCrop(244),
                                                             transforms.ToTensor(),
                                                             ]),
                            'target_transform': AsTensor()},
           'val_params': {'embed': [],
                          'image_dir': './data/ranzcr/train',
                          'targets_csv': './data/ranzcr/train.csv',
                          'target_type': 'ETT',
                          'transform': transforms.Compose([LoadImage(),
                                                           transforms.Resize(1500),
                                                           transforms.CenterCrop(1200),
                                                           transforms.Resize(300),
                                                           transforms.CenterCrop(244),
                                                           transforms.ToTensor(),
                                                          ]),
                            'target_transform': AsTensor()},
           'test_params': {'embed': [],
                           'image_dir': './data/ranzcr/train',
                           'targets_csv': './data/ranzcr/train.csv',
                           'target_type': 'ETT',
                           'transform': transforms.Compose([LoadImage(),
                                                            transforms.Resize(1500),
                                                            transforms.CenterCrop(1200),
                                                            transforms.Resize(300),
                                                            transforms.CenterCrop(244),
                                                            transforms.ToTensor(),
                                                            ]),
                           'target_transform': AsTensor()}}


model_params = {'model_name': 'resnet18',
                'in_channels': 1,
                'tv_params': {'num_classes': 3}} 
    
opt_params = {'lr': 0.01}
crit_params = {'reduction': 'sum'}
sample_params = {'set_seed': False,
                 'splits': (.7,.15)}
sched_params = {'factor': .5,
                'patience': 2,
                'cooldown': 2}

l = Learn([Ranzcr,Ranzcr,Ranzcr], tv_model, Selector, 
          Optimizer=Adam, Scheduler=ReduceLROnPlateau, Criterion=CrossEntropyLoss, 
          model_params=model_params, ds_params=ds_params, sample_params=sample_params,
          opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
          adapt=False, load_model=False, load_embed=False, save_model=True,
          batch_size=128, epochs=10)

In [ ]:
ds_params={'train_params': {'embed': [],
                            'image_dir': './data/ranzcr/train',
                            'targets_csv': './data/ranzcr/train.csv',
                            'target_type': 'ETT',
                            'transform': transforms.Compose([LoadImage(),
                                                             transforms.RandomRotation(10),
                                                             transforms.Resize((1500)),
                                                             transforms.CenterCrop(1200),
                                                             transforms.Resize((300)),
                                                             transforms.RandomCrop(244),
                                                             #transforms.ToTensor(),
                                                             #transforms.Normalize(0.3,0.3),
                                                             #transforms.ToPILImage(),
                                                             ]),
                            'target_transform': AsTensor()}}

ranzcr = Ranzcr(**ds_params['train_params'])

In [ ]:
ranzcr.ds_idx[0]

In [ ]:
ranzcr['86119358030110783442164990675803210852'][1]

In [ ]:
display(ranzcr['86119358030110783442164990675803210852'][0])

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/ranzcr/train.csv', header=0, index_col=0)

In [ ]:
df.shape

In [ ]:
pos = df.loc[df.iloc[:,10] == 1]
pos.shape
neg = df.loc[df.iloc[:,10] == 0].sample(pos.shape[0])
neg.shape


In [ ]:
neg = df.loc[df.iloc[:,10] == 0].sample(pos.shape[0])
neg.shape

In [ ]:
pos

In [ ]:
neg